In [1]:
import torch
import os
import transformers
from ast import literal_eval
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig

from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss

# 1. 설정: pandas 출력 옵션 및 시드 고정
pd.set_option('display.max_columns', None)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42)
# 2. 모델 및 토크나이저 로드 (8-bit 양자화)
model_name = "CarrotAI/Llama-3.2-Rabbit-Ko-3B-Instruct"

# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,            # 8-bit 정밀도로 모델 로드
#     llm_int8_threshold=6.0,       # int8 양자화 임계값
#     llm_int8_has_fp16_weight=False # FP16 가중치 사용 여부
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3. PEFT 설정 (LoRA)
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=['q_proj', 'v_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
# model.gradient_checkpointing_enable()


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.49s/it]


In [2]:
from datasets import Dataset
import pandas as pd
from ast import literal_eval

PROMPT_NO_QUESTION_PLUS = """
문제를 풀이할 때, 반드시 지문을 참고하세요.
문제는 무조건 1개의 정답만 있습니다.
문제를 풀이할 때 모든 선택지들을 검토하세요.
모든 선택지마다 근거를 지문에서 찾아 설명하세요.

다음의 형식을 따라 답변하세요.

1번: (선택지 1번에 대한 답변) + "(지문 속 근거가 된 문장)"
2번: (선택지 2번에 대한 답변) + "(지문 속 근거가 된 문장)"
3번: (선택지 3번에 대한 답변) + "(지문 속 근거가 된 문장)"
4번: (선택지 4번에 대한 답변) + "(지문 속 근거가 된 문장)"
5번: (선택지 5번에 대한 답변) + "(지문 속 근거가 된 문장)"
최종 정답: (근거 : 고른 정답에 대한 근거 # 정답 번호)

지문 : 
{paragraph}

질문 : 
{question}

선택지 : 
{choices}


1번, 2번, 3번, 4번, 5번 중에 하나를 정답으로 고르고, 근거와 고른 정답을 작성하세요.
근거와 정답은 "#"으로 구분하세요.
근거 :
"""

PROMPT_QUESTION_PLUS = """

문제를 풀이할 때, 반드시 지문을 참고하세요.
문제는 무조건 1개의 정답만 있습니다.
문제를 풀이할 때 모든 선택지들을 검토하세요.
모든 선택지마다 근거를 지문에서 찾아 설명하세요.

다음의 형식을 따라 답변하세요.
1번: (선택지 1번에 대한 답변) + "(지문 속 근거가 된 문장)"
2번: (선택지 2번에 대한 답변) + "(지문 속 근거가 된 문장)"
3번: (선택지 3번에 대한 답변) + "(지문 속 근거가 된 문장)"
4번: (선택지 4번에 대한 답변) + "(지문 속 근거가 된 문장)"
5번: (선택지 5번에 대한 답변) + "(지문 속 근거가 된 문장)"
최종 정답: (근거 : 고른 정답에 대한 근거 # 정답 번호)

지문 : 
{paragraph}
질문 : 
{question}
<보기> : 
{question_plus}
선택지 : 
{choices}

1번, 2번, 3번, 4번, 5번 중에 하나를 정답으로 고르고, 근거와 고른 정답을 작성하세요.
근거와 정답은 "#"으로 구분하세요.
근거 :

"""

dataset = pd.read_csv('datas/train+klue.csv')
# Load the train dataset
# TODO Train Data 경로 입력

# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('klue', None),
        "question_plus": problems.get('question_plus', None),
        'klue' : row.get('klue', None)
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)
dataset = Dataset.from_pandas(df)
processed_dataset = []

for i in range(len(dataset)):
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(dataset[i]["choices"])])

    # <보기>가 있을 때
    if dataset[i]["question_plus"]:
        user_message = PROMPT_QUESTION_PLUS.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            question_plus=dataset[i]["question_plus"],
            choices=choices_string,
        )
    # <보기>가 없을 때
    else:
        user_message = PROMPT_NO_QUESTION_PLUS.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            choices=choices_string,
        )

    # chat message 형식으로 변환
    processed_dataset.append(
        {
            "id": dataset[i]["id"],
            "messages": [
                {"role": "system", "content": """대한민국 수능 전문가이다. 수능 국어 지문과 질문 그리고 선택지가 주어졌을 때 정답을 맞출 수 있다."""},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": f"{dataset[i]['klue']}"}
            ],
            "label": dataset[i]['klue'],
        }
    )
print(processed_dataset[0])
processed_dataset = Dataset.from_pandas(pd.DataFrame(processed_dataset))
processed_dataset

{'id': 'yongari_generate_91578', 'messages': [{'role': 'system', 'content': '대한민국 수능 전문가이다. 수능 국어 지문과 질문 그리고 선택지가 주어졌을 때 정답을 맞출 수 있다.'}, {'role': 'user', 'content': '\n문제를 풀이할 때, 반드시 지문을 참고하세요.\n문제는 무조건 1개의 정답만 있습니다.\n문제를 풀이할 때 모든 선택지들을 검토하세요.\n모든 선택지마다 근거를 지문에서 찾아 설명하세요.\n\n다음의 형식을 따라 답변하세요.\n\n1번: (선택지 1번에 대한 답변) + "(지문 속 근거가 된 문장)"\n2번: (선택지 2번에 대한 답변) + "(지문 속 근거가 된 문장)"\n3번: (선택지 3번에 대한 답변) + "(지문 속 근거가 된 문장)"\n4번: (선택지 4번에 대한 답변) + "(지문 속 근거가 된 문장)"\n5번: (선택지 5번에 대한 답변) + "(지문 속 근거가 된 문장)"\n최종 정답: (근거 : 고른 정답에 대한 근거 # 정답 번호)\n\n지문 : \n상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服) 절차에 대하여 논한 것이 신과는 큰 차이가 있었습니다 . 장자를 위하여 3년을 입는 까닭은 위로 ‘정체(正體)’가 되기 때문이고 또 전 중(傳重: 조상의 제사나 가문의 법통을 전함)하기 때문입니다 . …(중략) … 무엇보다 중요한 것은 할아버지와 아버지의 뒤를 이은 ‘정체’이지, 꼭 첫째이기 때문에 참 최 3년 복을 입는 것은 아닙니다 .”라고 하였다 .－현종실록 －ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 .ㄷ.정조 시기에 탕평 정치의 한 축을 이루었다 .ㄹ.이 이와 성혼의 문인을 중심으로 형성되었다.\n\n질문 : \n상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?\n\n선택지 : \n1 - ㄱ, ㄴ\n2 - ㄱ, ㄷ\n3 - ㄴ, ㄹ\n4 - ㄷ, ㄹ\n

Dataset({
    features: ['id', 'messages', 'label'],
    num_rows: 2031
})

In [3]:

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example["messages"])):
        output_texts.append(
            tokenizer.apply_chat_template(
                example["messages"][i],
                tokenize=False,
            )
        )
    return output_texts

def tokenize(element):
    outputs = tokenizer(
        formatting_prompts_func(element),
        truncation=False,
        padding=False,
        return_overflowing_tokens=False,
        return_length=False,
    )
    return {
        "input_ids": outputs["input_ids"],
        "attention_mask": outputs["attention_mask"],
    }

# 데이터 토큰화
tokenized_dataset = processed_dataset.map(
    tokenize,
    remove_columns=list(processed_dataset.features),
    batched=True,
    num_proc=1,
    load_from_cache_file=True,
    desc="Tokenizing",
)

Tokenizing: 100%|██████████| 2031/2031 [00:04<00:00, 415.53 examples/s]


In [4]:
import numpy as np
# 데이터 분리
tokenized_dataset = tokenized_dataset.filter(lambda x: len(x["input_ids"]) <= 2048)  
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.10, seed=42)

train_dataset = tokenized_dataset['train']
eval_dataset = tokenized_dataset['test']


train_dataset_token_lengths = [len(train_dataset[i]["input_ids"]) for i in range(len(train_dataset))]
print(f"max token length: {max(train_dataset_token_lengths)}")
print(f"min token length: {min(train_dataset_token_lengths)}")
print(f"avg token length: {np.mean(train_dataset_token_lengths)}")

# 데이터 확인
print(tokenizer.decode(train_dataset[0]["input_ids"]))

Filter: 100%|██████████| 2031/2031 [00:01<00:00, 1114.52 examples/s]


max token length: 1886
min token length: 436
avg token length: 968.5900383141762
<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

대한민국 수능 전문가이다. 수능 국어 지문과 질문 그리고 선택지가 주어졌을 때 정답을 맞출 수 있다.<|eot_id|><|start_header_id|>user<|end_header_id|>

문제를 풀이할 때, 반드시 지문을 참고하세요.
문제는 무조건 1개의 정답만 있습니다.
문제를 풀이할 때 모든 선택지들을 검토하세요.
모든 선택지마다 근거를 지문에서 찾아 설명하세요.

다음의 형식을 따라 답변하세요.

1번: (선택지 1번에 대한 답변) + "(지문 속 근거가 된 문장)"
2번: (선택지 2번에 대한 답변) + "(지문 속 근거가 된 문장)"
3번: (선택지 3번에 대한 답변) + "(지문 속 근거가 된 문장)"
4번: (선택지 4번에 대한 답변) + "(지문 속 근거가 된 문장)"
5번: (선택지 5번에 대한 답변) + "(지문 속 근거가 된 문장)"
최종 정답: (근거 : 고른 정답에 대한 근거 # 정답 번호)

지문 : 
부산 강서구 김해공항 인근의 ‘공항마을’ 등 그린벨트(개발제한구역)에서 풀린 집단취락(마을) 지역에 상가나 공장을 지을 수 있게 된다. 또 임대주택을 35% 이상 의무적으로 짓도록 한 규제가 사실상 사라지는 등 그린벨트 내 규제가 대폭 완화된다. 국토교통부는 그린벨트 해제지역의 개발 사업을 활성화하기 위해 ‘개발제한구역의 조정을 위한 도시관리계획 변경안 수립 지침’ 및 ‘도시·군관리계획수립지침’을 이같이 개정해 11일부터 시행한다고 10일 발표했다. 개정된 지침은 그린벨트에서 풀린 집단취락이 시가지나 공항, 항만, 철도역 등 거점시설과 맞닿아 있는 경우 상가나 공장을 지을 수 있도록 했다. 기존에는 자연녹지지역이나 주거지역으로만 개발이 허용돼 정비사업이

In [5]:
response_template = "assistant<|end_header_id|>"
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)

In [6]:
import numpy as np
import evaluate
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
def preprocess_logits_for_metrics(logits, labels):
    logits = logits if not isinstance(logits, tuple) else logits[0]
    logit_idx = [tokenizer.vocab["1"],
                    tokenizer.vocab["2"],
                    tokenizer.vocab["3"],
                    tokenizer.vocab["4"], 
                    tokenizer.vocab["5"]]
    logits = logits[:, -2, logit_idx] # -2: answer token, -1: eos token
    return logits


    # metric 계산 함수
def compute_metrics(evaluation_result):
    logits, labels = evaluation_result
    int_output_map = {"1": 0, "2": 1, "3": 2, "4": 3, "5": 4}


    # 토큰화된 레이블 디코딩
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    labels = [label.split('#')[-1].strip() for label in labels]
    labels = [int_output_map.get(label, -1) for label in labels] 

    # 소프트맥스 함수를 사용하여 로그트 변환
    probs = torch.nn.functional.softmax(torch.tensor(logits, dtype=torch.float32), dim=-1)

    predictions = np.argmax(probs, axis=-1)

    # 정확도 계산
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")

    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}


In [7]:

sft_config = SFTConfig(
    output_dir=f"./outputs + {model_name.split('/')[-1]}",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=2,
    save_only_model=True,
    report_to="none",
    fp16 = True
)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics,
    args=sft_config,
    max_seq_length=2048,
    packing=False,
)



/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
# for param in model.parameters():
#     param.requires_grad = True

import gc
gc.collect()
torch.cuda.empty_cache()
trainer.train()

Epoch,Training Loss,Validation Loss


In [9]:
# Load the test dataset
# TODO Test Data 경로 입력
test_df = pd.read_csv('datas/test.csv')

# Flatten the JSON dataset
records = []
for _, row in test_df.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
test_df = pd.DataFrame(records)

In [10]:
from tqdm import tqdm
model.to('cuda:0')
test_dataset = []
for i, row in test_df.iterrows():
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(row["choices"])])
    len_choices = len(row["choices"])
    
    # <보기>가 있을 때
    if row["question_plus"]:
        user_message = PROMPT_QUESTION_PLUS.format(
            paragraph=row["paragraph"],
            question=row["question"],
            question_plus=row["question_plus"],
            choices=choices_string,
        )
    # <보기>가 없을 때
    else:
        user_message = PROMPT_NO_QUESTION_PLUS.format(
            paragraph=row["paragraph"],
            question=row["question"],
            choices=choices_string,
        )

    test_dataset.append(
        {
            "id": row["id"],
            "messages": [
            {"role": "system", "content": """대한민국 수능 전문가이다. 수능 국어 지문과 질문 그리고 선택지가 주어졌을 때 정답을 맞출 수 있다."""},
            {"role": "user", "content": user_message},
            ],
            "label": row["answer"],
            "len_choices": len_choices,
        }
    )

generated_infer_results = []
model.eval()
with torch.inference_mode():
    for idx, data in enumerate(tqdm(test_dataset)):
        _id = data["id"]
        messages = data["messages"]
        len_choices = data["len_choices"]

        # 텍스트 생성을 위한 입력 데이터 준비
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")
        # 모델을 이용한 텍스트 생성
        outputs = model.generate(
            inputs,
            max_new_tokens=200,  # 최대 생성 토큰 수
            pad_token_id=tokenizer.pad_token_id,
        )
        # 생성된 텍스트 디코딩
        generated_text = tokenizer.batch_decode(
            outputs[:,inputs.shape[1]:], skip_special_tokens=True)[0]
        generated_text
        # 생성된 텍스트와 라벨을 결과 리스트에 추가
        generated_infer_results.append({
            "id": _id,  # 고유 ID
            "answer": generated_text,  # 생성된 텍스트
            "label": data["label"]  # 실제 라벨이 있다면, data에서 가져옴
        })
generated_infer_results = pd.DataFrame(generated_infer_results)

  1%|          | 10/869 [00:35<49:40,  3.47s/it] 

100%|██████████| 869/869 [52:05<00:00,  3.60s/it] 


In [18]:
test = generated_infer_results['answer'].apply(lambda x: x.split('#')[-1])

In [26]:
test['len'] = test['answer'].apply(lambda x: len(x))

In [44]:
import re
def extract_last_number(text):
    numbers = re.findall(r'\d+', text) # 모든 숫자 추출
    if int(numbers[-1]) > 5:
        numbers = [1]
    return int(numbers[-1]) if numbers else None

generated_infer_results['answer'] = generated_infer_results['answer'].apply(extract_last_number)

In [45]:
generated_infer_results['answer'] = generated_infer_results['answer'].apply(extract_last_number)

TypeError: expected string or bytes-like object

In [62]:
generated_infer_results['answer'] = generated_infer_results['answer'].apply(lambda x: 1 if x > 5 else x)

In [67]:
generated_infer_results[['id', 'answer']].to_csv('output.csv',index = False)

In [68]:
pd.read_csv('output.csv')

,id,answer
0,generation-for-nlp-0,3
1,generation-for-nlp-1,2
2,generation-for-nlp-2,4
3,generation-for-nlp-3,1
4,generation-for-nlp-4,1
...,...,...
864,generation-for-nlp-1609,1
865,generation-for-nlp-1512,1
866,generation-for-nlp-1382,3
867,generation-for-nlp-702,3
